# Erros

A análise de erros é essencial para compreender as limitações dos algoritmos
numéricos e tentar garantir a precisão e estabilidade dos resultados fornecidos
por eles.

## $ \S 1 $ Tipos de erros

* __Erros de aproximação (ou arredondamento)__: O erro de aproximação ocorre
  quando um determinado número não pode ser representado exatamente no sistema
  de ponto flutuante, sendo portanto necessário arredondá-lo ou truncá-lo para
  que se ajuste ao formato especificado. Este tipo de erro é inerente ao sistema
  de ponto flutuante, pois decorre do número finito de dígitos significativos
  que são armazenados em memória. Os erros de aproximação podem se propagar e
  se acumular, levando potencialmente a imprecisões significativas no resultado
  final.

* __Erros de truncamento__: Os erros de truncamento são introduzidos quando um
  processo infinito, como uma seqüência ou método iterativo, é encerrado
  após um número finito de etapas. Por exemplo, ao calcular a representação
  de uma função por sua série de Taylor, os erros de truncamento surgem da
  omissão de termos de ordem superior. Estes erros dependem do algoritmo e do
  número de iterações ou termos utilizados. A redução dos erros de truncamento
  geralmente envolve o aumento do número de iterações ou termos, o que por
  outro lado pode levar também a um aumento do custo computacional e dos erros
  de arredondamento envolvidos.
  
* __Perda de significância__: A perda de significância ocorre quando dois números
  quase iguais são subtraídos, levando a uma perda de dígitos significativos no
  resultado. Este fenômeno também é conhecido como _cancelamento catastrófico_ e
  pode levar a imprecisões substanciais no cálculo. A perda de significância
  pode ser mitigada usando-se algoritmos ou técnicas que evitam ao máximo a
  subtração de números, ou aumentando-se a precisão da representação de ponto
  flutuante.

📝 Note que a palavra "truncamento" no segundo tipo não se refere ao modo de se
aproximar um dado número por um outro no formato de ponto flutuante (considerado
no caderno anterior), mas sim à omissão de um número passos ou termos de
uma expressão ou processo que nunca termina.


## $ \S 2 $ Exemplos de erros

__Exemplo 2.1 (erro de aproximação):__ 
Suponha que queiramos somar os números $ 0.1 $ e $ 0.2 $ em um sistema de ponto
flutuante como o padrão IEEE 754 de precisão simples. Ambos os números são
racionais, mas nenhum deles pode ser escrito como uma fração de inteiros cujo
denominador é potência de $ 2 $, logo suas representações na base binária são
infinitas. Por este motivo, necessariamente ocorrem erros de aproximação ao
tentarmos representá-los como números de ponto flutuante na máquina:

$$
0.1_{10} \approx 0.00011001100110011001101_{2} \quad \text{e} \quad 
0.2_{10} \approx 0.00110011001100110011001_{2}
$$

Ao somar esses dois números aproximados e converter o resultado de volta para
decimal obtemos:
$$
0.1_{10} + 0.2_{10} \approx 0.30000000000000004_{10} \quad \text{em vez de} \quad 0.3_{10} \,.
$$
O erro de arredondamento é portanto $ 4 \times 10^{-17} $.

In [1]:
x = 0.1
y = 0.2
x + y

0.30000000000000004

__Exemplo 2.2 (erro de truncamento):__ Suponha que desejemos calcular o valor da
constante $ e $. Uma idéia seria utilizar a série de Taylor da função
exponencial ao redor de $ 0 $, que converge para qualquer $ x $:
$$
e^x = \sum_{n=0}^{\infty} \frac{x^n}{n!} = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \cdots
$$
Contudo, pela natureza finita da máquina, é necessário truncar a série após um
número finito de termos. Se o truncamento for efetuado após os primeiros quatro
termos, obteremos o valor aproximado
$$
e = e^{1} \approx 1 + \frac{1}{1!} + \frac{1^2}{2!} + \frac{1^3}{3!} = \frac{8}{3} = 2.6\overline{6}\,.
$$

Já o valor real de $ e $ é aproximadamente $ 2.71828 $. O erro de truncamento é portanto
bastante significativo neste caso: $ 0.0516 $, aproximadamente.

__Exemplo 2.3 (perda de significância ou cancelamento catastrófico):__ 

Considere as somas seguintes no nosso sistema de ponto flutuante:
$$
3.40 + 9.87 \approx 13.0 \quad \text{e} \quad 3.40 + 100 \approx 103\,.
$$
Em ambos os casos, apesar do arredondamento, o resultado aproximado das operações está relativamente próximo do valor
exato. Agora considere a seguinte operação, que envolve uma subtração:
$$
(100 + 3.40) - 102 \approx 103 - 102 = 1\,.
$$
Aqui o valor exato é $ 1.4 $. Portanto o erro _relativo_
$$
\frac{1.4 - 1.0}{1.4}
$$
é de quase $ 30\% $. A explicação é que a subtração de dois números que estão
muito próximos um do outro pode gerar a perda de vários dígitos significativos.
O erro absoluto é pequeno, mas o relativo, que é o que importa na maioria dos
casos, não é.  Para tornar este ponto mais dramático, se continuando a conta do
nosso exemplo, tivéssemos que multiplicar o último resultado por $ 10\,000 $, o
resultado exato seria $ 14\,000 $, enquanto o valor aproximado seria apenas $
10\,000 $.


__Exemplo 6.4 (perda de significância ou cancelamento catastrófico):__
Consideremos outro exemplo da perda de significância.  O método mais simples
para se calcular a derivada $ f'(a) $ de uma função $ f $ num ponto $ a $
consiste em aproximá-la por
$$
\frac{f(a + h) - f(a)}{h}\,.
$$
Como $ f'(a) $ é por definição o limite desta quantidade conforme $ h \to 0 $,
é natural supor que quanto menor o valor de $ h $, melhor será a aproximação. Contudo,
há algumas dificuldades com esta idéia. Primeiramente, se $ h $ for muito
pequeno, ele pode ser substituído por zero no sistema de ponto flutuante, gerando
um erro na tentativa de efetuar a divisão. Independentemente disto, pode ocorrer
o cancelamento catastrófico no cálculo do numerador, que então é amplificado
pelo denominador muito pequeno.

Como ilustração vamos tentar calcular o valor da derivada do seno em $ a = 2 $,
usando este método para valores cada vez menores de $ h $.

In [2]:
from numpy import sin, cos


def derivative(f, x, h):
    return (f(x + h) - f(x)) / h


exact = cos(2)
for n in range(1, -20, -1):
    h = 10**(n)
    approx = derivative(sin, 2, h)
    print(f"expoente: {n:3}", f"\terro: {exact - approx: 3.6e}")

expoente:   1 	erro: -2.715598e-01
expoente:   0 	erro:  3.520306e-01
expoente:  -1 	erro:  4.473377e-02
expoente:  -2 	erro:  4.539514e-03
expoente:  -3 	erro:  4.545793e-04
expoente:  -4 	erro:  4.546418e-05
expoente:  -5 	erro:  4.546484e-06
expoente:  -6 	erro:  4.547462e-07
expoente:  -7 	erro:  4.485187e-08
expoente:  -8 	erro:  2.663398e-09
expoente:  -9 	erro:  5.817455e-08
expoente: -10 	erro:  2.802192e-07
expoente: -11 	erro: -1.940227e-06
expoente: -12 	erro:  7.577538e-05
expoente: -13 	erro:  1.867977e-04
expoente: -14 	erro:  5.737913e-03
expoente: -15 	erro: -8.307993e-02
expoente: -16 	erro: -4.161468e-01
expoente: -17 	erro: -4.161468e-01
expoente: -18 	erro: -4.161468e-01
expoente: -19 	erro: -4.161468e-01


## $ \S 3 $ Medidas do erro: erro absoluto e erro relativo

Seja $ x $ um número qualquer e denotemos por $ \bar x $ uma aproximação para $ x $.
Por exemplo, $ x $ pode ser um número irracional e $ \bar x $ aquele que o representa
no sistema de ponto flutuante da máquina. Ou $ x $ pode ser uma raiz de um polinômio
e $ \bar x $ uma aproximação para ela, obtida através de algum método numérico.
É importante ter uma medida da discrepância entre estes dois valores. Com isto em mente
definimos:
* O __erro absoluto__ associado a esta aproximação por 
  $$ \boxed{\varepsilon_x = x - \bar x} $$
* O __erro relativo__ associado a esta aproximação por 
  $$ \phantom{(x \ne 0)} \qquad \boxed{\rho_x = \frac{x - \bar x}{x} \qquad (x \ne 0) } $$

Em geral o valor exato $ x $ é desconhecido. Nestas situações o melhor que se pode
esperar é conseguir uma _cota_ para o erro relativo ou absoluto.

📝 Alguns autores preferem definir o erro absoluto por $ | x - \bar x |$ e o relativo
por $ \frac{| x - \bar x|}{| x |} $. De fato, na maior parte dos casos o que importa
é o tamanho do erro, não seu sinal.


__Teorema 3.1 (erro na adição e subtração):__
_Sejam $x$ e $y$ dois números reais, aproximados com erros
absolutos $\varepsilon_x$ e $\varepsilon_y$ e erros relativos $\rho_x$ e
$\rho_y$, respectivamente. Então, para a adição e subtração de $x$ e $y$, as
seguintes relações são válidas:_

(a) $ \varepsilon_{x\pm y} = \varepsilon_x + \varepsilon_y $\,.

(b) _Assumindo que_ $ x $, $ y $ _e_ $ x \pm y \ne 0 $,
$$
\rho_{x\pm y} = \frac{x}{x \pm y} \rho_x + \frac{y}{x \pm y}\rho_y
$$

__Prova:__ Vamos demonstrar o resultado para a soma de $ x $ e $ y $; o caso da diferença
é inteiramente análogo. Sejam $ \bar x $ e $ \bar y $ as respectivas aproximações.

(a) Pela definição de erro absoluto,
\begin{align*} \varepsilon_{x+ y} &= (x + y) - (\bar x + \bar y) \\
&= (x - \bar x) + (y - \bar y) \\
&= \varepsilon_x + \varepsilon_y \,.
\end{align*}

(b) Pela parte (a),
\begin{align*} \rho_{x+ y} &= \frac{\varepsilon_{x+ y}}{x + y} \\
&= \frac{\varepsilon_x + \varepsilon_y}{x + y} \\
&= \frac{x}{x + y}\, \frac{\varepsilon_x}{x} +
\frac{y}{x + y}\, \frac{\varepsilon_y}{y}\\
&= \frac{x}{x + y}\,\rho_x + \frac{y}{x + y}\,\rho_y\,.
\end{align*}


__Teorema 3.2 (erro no produto):__
_Sejam $x$ e $y$ números reais, aproximados por $ \bar x $ e
$ \bar y $, de modo que os respectivos erros absolutos e relativos sejam
$\varepsilon_x$, $\varepsilon_y$, $\rho_x$ e $\rho_y$. Suponha que os erros
relativos $ \rho_x $ e $ \rho_y $ sejam pequenos.  Então para o produto de $x$ e
$y$ valem as seguintes relações:_

_(a) O erro absoluto_ $ \varepsilon_{xy} = xy - \bar{x}\bar{y} $ _satisfaz_
$$ \varepsilon_{xy} \approx \bar{x}\,\varepsilon_y + \bar{y}\,\varepsilon_x + \varepsilon_x\,\varepsilon_y \,.$$

_(b) O erro relativo_ $ \rho_{xy} = \frac{xy - \bar{x}\bar{y}}{xy} $ _satisfaz_
$ \rho_{xy} \approx \rho_x + \rho_y \,$ _(assumindo que $ x,\, y \ne 0 $)_.

__Prova:__

(a) Pela definição do erro absoluto,
\begin{alignat*}{9}
\varepsilon_{xy} &= xy - \bar x \bar y \\
&= xy - x \bar y + x \bar y - \bar x \bar y  \\
&=  x(y - \bar y) + \bar y(x - \bar x)  \\
&= x\,\varepsilon_y + \bar y\,\varepsilon_x \\
&= x\,\varepsilon_y + \frac{\bar y}{y}\,y\,\varepsilon_x \\
&\approx  x\,\varepsilon_y + y\,\varepsilon_x \,.
\end{alignat*}
A última aproximação é tanto melhor quanto menor for o erro relativo $ \rho_y $. De fato,
$$
\left\lvert \frac{\bar y}{y} - 1 \right\rvert = 
\frac{|y - \bar y|}{| y |} = \rho_y\,. 
$$

(b) Como visto na demonstração da parte (a),
\begin{align*}
\rho_{xy} &= \frac{\varepsilon_{xy}}{xy} \\
&= \frac{x \, \varepsilon_y + \bar y \, \varepsilon_x}{xy} \\
&= \frac{\varepsilon_y}{y} + \frac{\bar y}{y}\, \frac{\varepsilon_x}{x} \\
&\approx \rho_x + \rho_y \,.
\end{align*}

__Teorema 3.3 (erro na divisão):__
_Sejam $x$ e $y$ dois números reais com erros absolutos
$\varepsilon_x = x - \bar{x}$ e $\varepsilon_y = y - \bar{y}$, e erros relativos
$\rho_x = \frac{x - \bar{x}}{x}$ e $\rho_y = \frac{y - \bar{y}}{y}$,
respectivamente, onde $ x,\,y \ne 0$. Então, para a divisão de $x$ por $y$, as
seguintes relações são válidas:_

(a) $ \varepsilon_{x / y} = \frac{x}{\bar y} \big( \rho_x - \rho_y \big)\,. $


(b) $ \rho_{x / y} \approx \rho_x - \rho_y\,. $

__Prova:__

(a) Temos
\begin{align*}
\varepsilon_{x / y} &= \frac{x}{y} - \frac{\bar{x}}{\bar{y}} \\
&= \frac{x\bar{y} - \bar x y}{y\bar y} \\
&= \frac{x\bar{y} - xy + xy - \bar x y}{y\bar y} \\
&= \frac{x\,(\bar{y} - y) + (x - \bar x)\,y}{y\bar y} \\
&= - \frac{\varepsilon_y}{y}\,\frac{x}{\bar y} + \frac{\varepsilon_x}{x}\,\frac{x}{\bar y} \\
&=\frac{x}{\bar y} \big( \rho_x - \rho_y \big)\,.
\end{align*}

(b) Da parte (a) segue que se $ \rho_y $ é pequeno, então:
\begin{align*} \rho_{x / y} &= \frac{\varepsilon_{x / y}}{x / y} \\
&=\frac{y}{\bar y}  \big( \rho_x - \rho_y \big) \\
&\approx \rho_x- \rho_y\,.
\end{align*}

📝 Note que em todos estes três teoremas, foram ignorados os erros de
aproximação envolvidos nos resultados $ \bar x \pm \bar y $, $ \bar x\bar y $ e 
$ \bar x / \bar y $, decorrentes do fato que estes números em geral não podem
ser representados fielmente no sistema de ponto flutuante. 